In [1]:
import codecs
import pandas as pd
import numpy as np
import re
#正则表达式

In [2]:
def origin2tag():
    input_data=codecs.open('BosonNLP_NER_6C.txt','r','utf-8')
    output_data=codecs.open('./wordtag.txt','w','utf-8')
    for line in input_data.readlines():
        line=line.strip()
        i=0
        while i<len(line):
            if line[i]=='{':
                i+=2
                temp=''
                while line[i]!='}':
                    temp+=line[i]
                    i+=1
                i+=2
                word=temp.split(':')
                sen=word[1]
                output_data.write(sen[0]+'/B_'+word[0]+' ')
                for j in sen[1:len(sen)-1]:
                    output_data.write(j+"/M_"+word[0]+" ")
                output_data.write(sen[-1]+"/E_"+word[0]+" ")
            else:
                output_data.write(line[i]+"/O ")
                i+=1
        output_data.write('\n')
    input_data.close()
    output_data.close()

In [3]:
#{{product_name:浙江在线杭州}}{{time:4月25日}}讯（记者{{person_name: 施宇翔}} 通讯员 {{person_name:方英}}）毒贩很“时髦”，用{{product_name:微信}}交易毒品。没料想警方也很“潮”，将计就计，一举将其擒获。
#浙/B_product_name 江/M_product_name 在/M_product_name 线/M_product_name 杭/M_product_name 州/E_product_name 4/B_time 月/M_time 2/M_time 5/M_time 日/E_time 讯/O （/O 记/O 者/O  /B_person_name 施/M_person_name 宇/M_person_name 翔/E_person_name  /O 通/O 讯/O 员/O  /O 方/B_person_name 英/E_person_name ）/O 毒/O 贩/O 很/O “/O 时/O 髦/O ”/O ，/O 用/O 微/B_product_name 信/E_product_name 交/O 易/O 毒/O 品/O 。/O 没/O 料/O 想/O 警/O 方/O 也/O 很/O “/O 潮/O ”/O ，/O 将/O 计/O 就/O 计/O ，/O 一/O 举/O 将/O 其/O 擒/O 获/O 。

In [7]:
def data2pkl():
    datas = list()
    labels = list()
    linedata=list()
    linelabel=list()
    tags = set()
    input_data = codecs.open('./wordtagsplit.txt','r','utf-8')
    for line in input_data.readlines():
        line = line.split()
        linedata=[]
        linelabel=[]
        num=0
        for word in line:
            word = word.split('/')
            linedata.append(word[0])
            linelabel.append(word[1])
            tags.add(word[1])
            if word[1]!='O':
                num=1
        if num!=0:
            datas.append(linedata)
            labels.append(linelabel)
    input_data.close()
    import collections
    def flatten(x):    
        result = []    
        for el in x:        
            if isinstance(x, collections.Iterable) and not isinstance(el, str):            
                result.extend(flatten(el))        
            else:            
                result.append(el)    
        return result

    
    all_words = flatten(datas)
    #all_words=list(filter(None,all_words))#去空值
    sr_allwords = pd.Series(all_words)
    sr_allwords = sr_allwords.value_counts()
    set_words = sr_allwords.index
    set_ids = range(1, len(set_words)+1)

    tags = [i for i in tags]
    #
    tag_ids = range(len(tags))
    word2id = pd.Series(set_ids, index=set_words)
    id2word = pd.Series(set_words, index=set_ids)
    tag2id = pd.Series(tag_ids, index=tags)
    id2tag = pd.Series(tags, index=tag_ids)
    word2id["unknow"] = len(word2id)+1
    print(len(tags))
    print (word2id)
    print(tag2id)
    #建立一个word2id词典，把每个汉字转换成id。这里习惯现按照数据集中每个汉字出现的次数排序，id从1开始
    #再建立一个tag2id词典，把每一个字标注的类型转换成id
    
    max_len = 60
    def X_padding(words):
        ids = list(word2id[words])
        if len(ids) >= max_len:  
            return ids[:max_len]
        ids.extend([0]*(max_len-len(ids))) 
        return ids
    def y_padding(tags):
        ids = list(tag2id[tags])
        if len(ids) >= max_len: 
            return ids[:max_len]
        ids.extend([0]*(max_len-len(ids))) 
        return ids

    df_data = pd.DataFrame({'words': datas, 'tags': labels}, index=range(len(datas)))
    df_data['x'] = df_data['words'].apply(X_padding)
    df_data['y'] = df_data['tags'].apply(y_padding)
    x = np.asarray(list(df_data['x'].values))
    y = np.asarray(list(df_data['y'].values))
    #print(x)
    #print(y)

    from sklearn.model_selection import train_test_split
    x_train,x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=43)
    x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train,  test_size=0.2, random_state=43)#训练集抽一部分验证
    
    
    import pickle
    import os
    with open('Bosondata.pkl', 'wb') as outp:
        pickle.dump(word2id, outp)
        pickle.dump(id2word, outp)
        pickle.dump(tag2id, outp)
        pickle.dump(id2tag, outp)
        pickle.dump(x_train, outp)
        pickle.dump(y_train, outp)
        pickle.dump(x_test, outp)
        pickle.dump(y_test, outp)
        pickle.dump(x_valid, outp)
        pickle.dump(y_valid, outp)
    print ('** Finished saving the data.')
#把刚按标点分开的数据，按照一一对应的顺序，把汉字和每个字的标签转换成id，分别存到两个数组里面，
#一起保存到一个pkl文件中，这样模型使用时候就可以直接读取，不用每次都处理数据

In [8]:
def tagsplit():
    with open('wordtag.txt','rb') as inp:
        texts=inp.read().decode('utf-8')
    sentences=re.split('[，。！？、‘’“”（）]/[O]',texts)
    output_data=codecs.open('wordtagsplit.txt','w','utf-8')
    for sentence in sentences:
        if sentence!='':
            output_data.write(sentence.strip()+'\n')
    output_data.close

In [9]:
#origin2tag()
#tagsplit()
data2pkl()

23
的            1
1            2
0            3
             4
2            5
在            6
中            7
国            8
年            9
一           10
了           11
日           12
是           13
月           14
大           15
为           16
人           17
3           18
上           19
有           20
5           21
行           22
n           23
市           24
发           25
到           26
时           27
来           28
和           29
会           30
          ... 
诟         3406
辟         3407
黯         3408
贿         3409
屡         3410
記         3411
捡         3412
锂         3413
哑         3414
锌         3415
項         3416
窍         3417
纺         3418
後         3419
喇         3420
缅         3421
堰         3422
５         3423
斓         3424
铠         3425
堕         3426
鄙         3427
琥         3428
埂         3429
菏         3430
崖         3431
菁         3432
肿         3433
侉         3434
unknow    3435
Length: 3435, dtype: int64
E_org_name         0
                   1
E_ime        

In [12]:
df_data

,words,tags,x,y
0,"[浙, 江, 在, 线, 杭, 州, 4, 月, 2, 5, 日, 讯]","[B_product_name, M_product_name, M_product_nam...","[529, 192, 6, 313, 522, 156, 32, 14, 5, 21, 12...","[15, 13, 13, 13, 13, 7, 8, 19, 19, 19, 4, 17, ..."
1,"[记, 者, , 施, 宇, 翔, , 通, 讯, 员, , 方, 英]","[O, O, B_person_name, M_person_name, M_person_...","[132, 80, 4, 702, 1294, 1582, 4, 111, 184, 134...","[17, 17, 1, 6, 6, 16, 17, 17, 17, 17, 17, 1, 1..."
2,"[用, 微, 信, 交, 易, 毒, 品]","[O, B_product_name, E_product_name, O, O, O, O]","[81, 315, 206, 249, 376, 1274, 101, 0, 0, 0, 0...","[17, 15, 7, 17, 17, 17, 17, 0, 0, 0, 0, 0, 0, ..."
3,"[记, 者, 从, 杭, 州, 江, 干, 区, 公, 安, 分, 局, 了, 解, 到]","[O, O, O, B_org_name, M_org_name, M_org_name, ...","[132, 80, 160, 522, 156, 192, 874, 109, 38, 14...","[17, 17, 17, 18, 21, 21, 21, 21, 21, 21, 21, 1..."
4,"[江, 干, 区, 禁, 毒, 专, 案, 组, 抓, 获, 吸, 贩, 毒, 人, 员, ...","[B_org_name, M_org_name, M_org_name, M_org_nam...","[192, 874, 109, 1656, 1274, 389, 428, 457, 907...","[18, 21, 21, 21, 21, 21, 21, 11, 17, 17, 17, 1..."
5,"[黑, 龙, 江, 籍, 男, 子, 钱, 某, 长, 期, 落, 脚, 于, 宾, 馆]","[B_location, M_location, E_location, O, O, O, ...","[1055, 558, 192, 1165, 489, 98, 707, 78, 72, 1...","[12, 5, 20, 17, 17, 17, 1, 16, 17, 17, 17, 17,..."
6,"[经, 常, 半, 夜, 驾, 车, 来, 往, 于, 杭, 州, 主, 城, 区, 的, ...","[O, O, O, O, O, O, O, O, O, B_location, M_loca...","[59, 406, 483, 1020, 995, 42, 28, 617, 48, 522...","[17, 17, 17, 17, 17, 17, 17, 17, 17, 12, 5, 5,..."
7,"[并, 且, 常, 要, 活, 动, 到, 凌, 晨, 6, 、, 7, 点, 钟]","[O, O, O, O, O, O, O, B_time, M_time, M_time, ...","[251, 841, 406, 120, 356, 66, 26, 1026, 824, 3...","[17, 17, 17, 17, 17, 17, 17, 8, 19, 19, 19, 19..."
8,"[白, 天, 则, 在, 家, 里, 呼, 呼, 大, 睡]","[B_time, E_time, O, O, O, O, O, O, O, O]","[436, 44, 623, 6, 40, 217, 1474, 1474, 15, 206...","[8, 4, 17, 17, 17, 17, 17, 17, 17, 17, 0, 0, 0..."
9,"[钱, 某, 不, 寻, 常, 的, 特, 征]","[B_person_name, E_person_name, O, O, O, O, O, O]","[707, 78, 33, 1172, 406, 1, 218, 1197, 0, 0, 0...","[1, 16, 17, 17, 17, 17, 17, 17, 0, 0, 0, 0, 0,..."
